## Backtest Strategy Playground

Notebook ini dipakai untuk mencoba berbagai strategy dari folder `src/strategy_backtest/strategies`.
Ganti nilai `STRATEGY_NAME` pada sel konfigurasi untuk memanggil file strategy yang berbeda.

In [1]:
from __future__ import annotations

import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython import get_ipython
from IPython.display import display

PROJECT_ROOT = Path('..').resolve()
for path in (PROJECT_ROOT, PROJECT_ROOT / 'src'):
    if str(path) not in sys.path:
        sys.path.append(str(path))

_ip = get_ipython()
if _ip is not None:
    try:
        _ip.run_line_magic('matplotlib', 'inline')
    except AttributeError:
        plt.switch_backend('Agg')
else:
    plt.switch_backend('Agg')

plt.style.use('seaborn-v0_8-darkgrid')

from src.strategy_backtest import (
    SignalBacktester,
    get_strategy,
    list_strategies,
    load_strategy_csv,
)

In [2]:
# Konfigurasi dataset & strategy
DATA_FILE = PROJECT_ROOT / 'data' / 'OKX_ETHUSDT.P, 1D.csv'
TIME_COLUMN = 'time'
PRICE_COLUMN = 'close'
ASSET_SYMBOL = 'ETHUSDT'

# Ganti nama strategy sesuai file di `src/strategy_backtest/strategies/`
STRATEGY_NAME = 'vwap'
# Opsional: override parameter default strategy
STRATEGY_PARAMS = {}

In [3]:
data, column_mapping = load_strategy_csv(DATA_FILE, time_column=TIME_COLUMN)
print(f'Dataset berisi {len(data):,} bar dengan {len(data.columns)} kolom.')
print('Contoh mapping kolom (sanitised -> original):')
for alias, original in list(column_mapping.items())[:10]:
    print(f'  {alias} -> {original}')

display(data.head())

Dataset berisi 2,152 bar dengan 35 kolom.
Contoh mapping kolom (sanitised -> original):
  time -> time
  open -> open
  high -> high
  low -> low
  close -> close
  volume -> Volume
  vwap -> VWAP
  vwap_1 -> VWAP.1
  ema -> EMA
  lucid_connector -> LUCID Connector


,open,high,low,close,volume,vwap,vwap_1,ema,lucid_connector,hyperwave,...,upper_confluence_zone,lower_confluence_zone,confluence_meter_value,custom_alert_condition_highlighter,alert_scripting_condition_highlighter,at_valuewhen,atr,histogram,macd,signal
time,,,,,,,,,,,,,,,,,,,,,
2019-12-25,127.64,127.72,123.40,125.07,2.193667e+06,125.396667,125.396667,NaN,0,NaN,...,105,-5,28.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-26,125.07,132.00,124.37,125.61,3.770336e+06,127.326667,127.326667,NaN,0,NaN,...,105,-5,28.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-27,125.66,126.96,122.65,126.26,5.765802e+06,125.290000,125.290000,NaN,0,NaN,...,105,-5,28.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-28,126.24,129.68,125.86,128.09,6.235318e+06,127.876667,127.876667,NaN,0,NaN,...,105,-5,28.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-29,128.08,137.97,127.57,134.26,6.826287e+06,133.266667,133.266667,NaN,0,NaN,...,105,-5,28.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
available = list_strategies()
print('Strategi tersedia:', ', '.join(available))
strategy = get_strategy(STRATEGY_NAME, **STRATEGY_PARAMS)
print('Deskripsi strategi:')
print(f"- Nama: {strategy.metadata.name}")
print(f"- Deskripsi: {strategy.metadata.description}")
print(f"- Entry: {strategy.metadata.entry}")
print(f"- Exit: {strategy.metadata.exit}")
print('Parameter default:')
for key, value in strategy.metadata.parameters.items():
    print(f'  {key}: {value}')
if not strategy.metadata.parameters:
    print('  (tidak ada parameter default eksplisit)')
print('Parameter aktif:')
for key, value in strategy.params.items():
    print(f'  {key}: {value}')
if not strategy.params:
    print('  (menggunakan nilai default)')

signals = strategy.generate_signals(data)
print('Kolom sinyal:', list(signals.columns))
display(signals.head())

Strategi tersedia: ema112_atr, vwap
Deskripsi strategi:
- Nama: vwap
- Deskripsi: Strategi VWAP yang mencari pantulan counter-trend di bawah VWAP dan fade rally di atas VWAP saat tren turun.
- Entry: Long ketika harga pertama kali menyelam di bawah VWAP dan ada konfirmasi bullish (wick bawah, volume naik, RSI > 30) untuk potensi reversion. Short ketika harga memantul di atas VWAP dalam tren turun dengan candle rejection, volume melemah, RSI < 70 dan MACD histogram melemah.
- Exit: Keluar utama di level VWAP sebagai target reversion. Stop-loss menggunakan ATR. Jika target VWAP tercapai, posisi ditutup penuh.
Parameter default:
  rsi_window: 14
  volume_ma_window: 20
  volume_spike_ratio: 1.1
  volume_fade_ratio: 0.9
  atr_window: 14
  atr_stop_multiplier: 1.5
  min_reversion_atr: 0.5
  session_frequency: 1D
  macd_fast: 12
  macd_slow: 26
  macd_signal: 9
  wick_ratio_threshold: 0.6
  allow_counter_trend_long: True
  allow_trend_short: True
Parameter aktif:
  rsi_window: 14
  volume_ma_

,long_entry,long_exit,short_entry,short_exit,vwap,rsi,macd,macd_signal,macd_hist,atr,volume_ma,active_entry_price,stop_level,target_level,exit_flag,position
time,,,,,,,,,,,,,,,,
2019-12-25,False,False,False,False,125.396667,NaN,0.000000,0.000000,0.000000,4.320000,2.193667e+06,NaN,NaN,125.396667,<NA>,flat
2019-12-26,False,False,False,False,127.326667,NaN,0.043077,0.008615,0.034462,4.556429,2.982002e+06,NaN,NaN,127.326667,<NA>,flat
2019-12-27,False,False,False,False,125.290000,NaN,0.128188,0.032530,0.095658,4.538827,3.909935e+06,NaN,NaN,125.290000,<NA>,flat
2019-12-28,False,False,False,False,127.876667,NaN,0.339392,0.093902,0.245490,4.487482,4.491281e+06,NaN,NaN,127.876667,<NA>,flat
2019-12-29,False,False,False,False,133.266667,NaN,0.993191,0.273760,0.719431,4.909805,4.958282e+06,NaN,NaN,133.266667,<NA>,flat


In [5]:
backtester = SignalBacktester(data=data, price_column=PRICE_COLUMN)
outputs = backtester.run(signals)

print('Metrik performa:')
for key, value in outputs.metrics.items():
    if isinstance(value, (int, float, np.floating)):
        print(f'- {key}: {value:.4f}')
    else:
        print(f'- {key}: {value}')

print('Ringkasan trade:')
for key, value in outputs.trade_summary.items():
    print(f'- {key}: {value}')

display(outputs.trades.head())

Metrik performa:
- total_return: 0.3869
- cagr: 0.0390
- sharpe_ratio: 0.4055
- annualised_vol: 0.1089
- max_drawdown: -0.2377
- avg_drawdown_duration: 237.8333
Ringkasan trade:
- total_trades: 46
- long_trades: 31
- short_trades: 15
- win_rate: 0.5869565217391305
- avg_pnl_pct: 0.007832702128946102
- median_bars: 1.0


,trade_id,direction,entry_time,exit_time,entry_price,exit_price,pnl_pct,pnl_currency,bars_held,exit_reason,...,exit_macd,exit_macd_signal,exit_macd_hist,exit_atr,exit_volume_ma,exit_active_entry_price,exit_stop_level,exit_target_level,exit_exit_flag,exit_position
0,1,Short,2020-03-21,2020-03-22,132.63,122.40,0.077132,10.23,1,short_exit_signal:target_vwap,...,-27.306804,-25.594723,-1.712081,22.605884,1.008972e+08,132.63,167.274505,131.970000,target_vwap,short
1,2,Long,2020-04-23,2020-04-24,185.55,187.54,0.010725,1.99,1,long_exit_signal:target_vwap,...,7.838437,5.309711,2.528726,13.058972,7.330002e+07,185.55,165.038584,185.930000,target_vwap,long
2,3,Long,2020-05-29,2020-05-30,220.65,243.68,0.104373,23.03,1,long_exit_signal:target_vwap,...,7.458373,5.010562,2.447810,12.691413,5.312812e+07,220.65,203.485410,221.216667,target_vwap,long
3,4,Short,2020-06-20,2020-06-21,228.72,228.09,0.002754,0.63,1,short_exit_signal:target_vwap,...,2.401917,5.072495,-2.670579,10.168785,7.166587e+07,228.72,244.657268,228.366667,target_vwap,short
4,5,Long,2020-07-26,2020-07-28,310.97,316.74,0.018555,5.77,2,long_exit_signal:target_vwap,...,20.112442,11.768365,8.344077,14.177469,1.399186e+08,310.97,291.603012,322.393333,target_vwap,long


In [6]:
def _as_bool(series: pd.Series | None) -> pd.Series:
    if series is None:
        return pd.Series(False, index=data.index)
    aligned = series.reindex(data.index)
    return aligned.fillna(False).astype(bool)

close_prices = data[PRICE_COLUMN]
long_entries = _as_bool(signals.get('long_entry'))
long_exits = _as_bool(signals.get('long_exit'))
short_entries = _as_bool(signals.get('short_entry'))
short_exits = _as_bool(signals.get('short_exit'))

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(close_prices.index, close_prices, label='Close', color='black', linewidth=1.2)

if long_entries.any():
    ax.scatter(close_prices.index[long_entries], close_prices[long_entries], marker='^', color='green', label='Long Entry', zorder=5)
if short_entries.any():
    ax.scatter(close_prices.index[short_entries], close_prices[short_entries], marker='v', color='red', label='Short Entry', zorder=5)
if long_exits.any():
    ax.scatter(close_prices.index[long_exits], close_prices[long_exits], marker='v', color='tab:blue', label='Long Exit', zorder=6)
if short_exits.any():
    ax.scatter(close_prices.index[short_exits], close_prices[short_exits], marker='^', color='tab:orange', label='Short Exit', zorder=6)

ax.set_title(f'{ASSET_SYMBOL} Close dengan Sinyal {STRATEGY_NAME}')
ax.set_ylabel('Harga')
ax.legend(loc='upper left', ncol=2)
fig.tight_layout()
display(fig)
plt.close(fig)

<Figure size 1400x600 with 1 Axes>

In [7]:
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(outputs.results.index, outputs.results['equity_curve'], color='C4', label='Equity Curve')
ax.set_title('Equity Curve Strategi')
ax.set_ylabel('Notional')
ax.legend()
fig.tight_layout()
display(fig)
plt.close(fig)

display(outputs.results.head())

<Figure size 1400x400 with 1 Axes>

,close,asset_return,position,strategy_return,equity_curve,drawdown,cumulative_pnl
time,,,,,,,
2019-12-25,125.07,0.000000,0.0,0.0,1.0,0.0,0.0
2019-12-26,125.61,0.004318,0.0,0.0,1.0,0.0,0.0
2019-12-27,126.26,0.005175,0.0,0.0,1.0,0.0,0.0
2019-12-28,128.09,0.014494,0.0,0.0,1.0,0.0,0.0
2019-12-29,134.26,0.048169,0.0,0.0,1.0,0.0,0.0
